## Módulo 14 - K-Means

In [64]:
# EDA
import pandas as pd
import numpy as np
import plotly.express as px

# Estatistics
from scipy.stats import bartlett, shapiro
from pingouin import welch_anova

# ML
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, pairwise_distances
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer

# HP otimization
import optuna

# generate model
import joblib

# create app batch
import gradio as gr

In [65]:
# padronização geral do formato de arrendondamento numéricos
pd.set_option('display.float_format', lambda x:'%.2f' % x)

### Bloco 1 - Análise Exploratória de Dados

#### Setup e Carga de Dados

Neste vídeo, vamos começar a trabalhar com o algoritmo K-Means no contexto de clusterização de clientes. O objetivo é agrupar os clientes em clusters com base em suas características, para oferecer um atendimento personalizado. Vou criar um arquivo Jupyter Notebook chamado "clusterização-clientes.ipynb" e importar as bibliotecas necessárias, como Pandas, Plotly, Scikit-Learn, Optuna, entre outras. Também vamos utilizar algumas métricas específicas para avaliar os resultados da clusterização. Além disso, faremos a carga dos dados de clientes a partir de um arquivo CSV e faremos uma análise inicial da estrutura desses dados. No próximo vídeo, continuaremos com o desenvolvimento do projeto.

In [66]:
df_clients = pd.read_csv(filepath_or_buffer='../datasets/raw/dataset_clientes-pj.csv')
df_clients.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 6 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   atividade_economica     500 non-null    object 
 1   faturamento_mensal      500 non-null    float64
 2   numero_de_funcionarios  500 non-null    int64  
 3   localizacao             500 non-null    object 
 4   idade                   500 non-null    int64  
 5   inovacao                500 non-null    int64  
dtypes: float64(1), int64(3), object(2)
memory usage: 23.6+ KB


In [67]:
df_clients.head()

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao
0,Comércio,713109.95,12,Rio de Janeiro,6,1
1,Comércio,790714.38,9,São Paulo,15,0
2,Comércio,1197239.33,17,São Paulo,4,9
3,Indústria,449185.78,15,São Paulo,6,0
4,Agronegócio,1006373.16,15,São Paulo,15,8


#### EDA

Neste vídeo, vamos dar continuidade ao nosso projeto no Jupyter Notebook, focando agora na Análise Exploratória de Dados (EDA). A ideia aqui não é repetir o EDA que já fizemos nos módulos anteriores, mas sim explorar algumas técnicas novas, como testes estatísticos. Começaremos analisando a distribuição da variável "inovação" em nosso conjunto de dados. Em seguida, levantaremos a hipótese de que empresas com diferentes níveis de inovação possam ter médias de faturamento mensal distintas. Para testar essa hipótese, utilizaremos o teste estatístico ANOVA (Análise de Variância), que verifica se há variações significativas nas médias de diferentes grupos. No entanto, antes de aplicar o teste, precisamos validar algumas suposições, como a independência das observações, a normalidade dos dados e a homogeneidade das variâncias. Caso essas suposições não sejam atendidas, utilizaremos o teste de ANOVA de Welch, que é mais robusto nessas situações. Nos próximos vídeos, veremos como realizar essas validações e aplicar o teste de hipótese.

In [68]:
# distribuição de variável inovação
inovation_distribution = df_clients.value_counts(subset='inovacao') / len(df_clients) * 100

# `.index` -> uma cor para cada coluna (feature)
px.bar(data_frame=inovation_distribution, color=inovation_distribution.index)

#### Introdução ANOVA e Testes Estatísticos

Nesta aula, vamos analisar os pressupostos necessários para realizar um teste estatístico de ANOVA. Vamos verificar se as variâncias entre os grupos são homogêneas e se os dados seguem uma distribuição normal. Também vamos considerar que as observações são independentes e que a variável independente é contínua. Além disso, vamos observar que as amostras não têm tamanhos iguais. Para verificar a homogeneidade das variâncias, vamos utilizar o teste de Bartlett. Para verificar a distribuição normal, vamos utilizar o teste de Shapiro-Wilk. Com base nos resultados desses testes, poderemos determinar se os pressupostos foram atendidos e prosseguir com o teste de ANOVA adequado.

**Teste ANOVA (Análise de Variância)**: Verificar se há variações significativas na média de faturamento mensal para diferentes níveis de inovação

*Suposições Pressupostos:*
- Observações independentes
- Variável dependente é contínua
- segue uma distribuição normal
- homogeneidade das variâncias
- Amostras sejam de tamanhos iguais

In [69]:
# checar se as variâncias (faturamento) entre os grupos (inovação) são homogêneas
# aplicar Teste de Barlett
# H0 - Variâncias são iguais
# H1 Variâcias não são iguais

# Separando dados de faturamento em grupo de classes na coluna "inovação"
revenue_group_by_inovation = [df_clients['faturamento_mensal']
                              [df_clients['inovacao'] == group] for group in df_clients['inovacao'].unique()]

# executar teste de Barlett
bartlett_test_statistic, bartlett_p_value = bartlett(*revenue_group_by_inovation)

print(f'Teste de Barlett:\n- Estatística: {
      bartlett_test_statistic:.4f}\n- P_value: {bartlett_p_value:.4f}')
print("Como o p_value > 0.05, não podemos podemos rejeitar H0")

Teste de Barlett:
- Estatística: 10.9012
- P_value: 0.2825
Como o p_value > 0.05, não podemos podemos rejeitar H0


In [70]:
# Executar Teste de Shapiro_Wilk
# Verificar se os dados seguem um distribuição normal
# H0 - Segue uma distruibuição normal
# H1 - Não segue uma distruibição normal

# executar teste de Barlett
shapiro_test_statistic, shapiro_p_value = shapiro(df_clients['faturamento_mensal'])

print(f'Teste de SW:\n- Estatística: {
      shapiro_test_statistic:.4f}\n- p_value: {shapiro_p_value:.4f}')
print("Como o p_value > 0.05, não podemos podemos rejeitar H0")

Teste de SW:
- Estatística: 0.9960
- p_value: 0.2351
Como o p_value > 0.05, não podemos podemos rejeitar H0


#### Teste de ANOVA

Neste vídeo, concluímos a parte de EDA (Exploratory Data Analysis) com um teste de ANOVA para avaliar a relação entre as médias dos valores para cada nível de inovação. Utilizamos o teste de ANOVA de Welch, pois as amostras têm tamanhos diferentes. Para isso, utilizamos o pacote estatístico PINGUIM e importamos o módulo Welch ANOVA. O resultado do teste mostrou que não há diferenças significativas entre as médias dos grupos, independentemente do nível de inovação das empresas. Isso significa que empresas com diferentes níveis de inovação podem ter faturamentos semelhantes. No próximo vídeo, exploraremos o algoritmo de K-Means.

In [71]:
aov = welch_anova(dv='faturamento_mensal', between='inovacao', data=df_clients)

print(f'Teste de ANOVA Welsh:\n- Estatística: {
      aov.loc[0, "F"]:.4f}\n- p_value: {aov.loc[0,'p-unc']:.4f}')
print("Como o p_value > 0.05, não podemos podemos rejeitar H0")

Teste de ANOVA Welsh:
- Estatística: 1.1270
- p_value: 0.3453
Como o p_value > 0.05, não podemos podemos rejeitar H0


### Bloco 2 - Treinamento do Modelo

#### Preparação dos Dados para o KMeans

Neste vídeo, começamos a trabalhar com o algoritmo de K-Means. Para treinar o algoritmo, precisamos preparar os dados. Faremos transformações nas colunas para que estejam no formato correto. Selecionaremos todas as colunas, pois não temos um target específico. Separaremos as variáveis numéricas, categóricas e ordinais. Aplicaremos transformações em cada tipo de variável usando um standard scaler, one-hot-encoder e ordinal encoder. Em seguida, criaremos um pré-processador que transformará os dados em um novo dataset. Usaremos o column transformer para indicar quais transformadores usar. Por fim, faremos o fit transform no pré-processador para treinar os transformadores e aplicar as transformações nos dados.

In [72]:
# PIPELINE DE TRANSFORMAÇÃO

dataframe_copy = df_clients.copy()

# selecionar colunas para clusterização
numeric_features = ['faturamento_mensal', 'numero_de_funcionarios', 'idade']
categorical_features = ['localizacao', 'atividade_economica']
ordinal_features = ['inovacao']

# Aplicar transformações do tipo
numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()
ordinal_transformer = OrdinalEncoder()

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('ord', ordinal_transformer, ordinal_features),
    ]
)

# transformar os dados
df_clients_transformed = preprocessor.fit_transform(X=dataframe_copy)

# Extraindo os nomes das colunas para as variáveis categóricas após o ajuste
categorical_names = preprocessor.named_transformers_['cat'].categories_
cat_column_names = np.concatenate(categorical_names).ravel()

# Concatenando os nomes das colunas
column_names = numeric_features + list(cat_column_names) + ordinal_features

# Exibindo os dados transformados de forma mais amigável
def display_transformed_data(data, column_names) -> None:
    print("".join([f"{name:<30}" for name in column_names]))
    for row in data[:8]:  # exibir apenas os primeiros 8 registros
        print("".join([f"{value:<30.2f}" if isinstance(
            value, float) else f"{value:<30}" for value in row]))


display_transformed_data(data=df_clients_transformed, column_names=column_names)

faturamento_mensal            numero_de_funcionarios        idade                         Belo Horizonte                Rio de Janeiro                São Paulo                     Vitória                       Agronegócio                   Comércio                      Indústria                     Serviços                      inovacao                      
-0.75                         -0.54                         -1.10                         0.00                          1.00                          0.00                          0.00                          0.00                          1.00                          0.00                          0.00                          1.00                          
-0.56                         -1.50                         1.94                          0.00                          0.00                          1.00                          0.00                          0.00                          1.00                          0.00    

#### Executando o KMeans com Optuna

Neste vídeo, vamos preparar os dados para treinar nosso modelo de algoritmo. Vamos otimizar a métrica de silhueta usando o Optuna para ajustar os hiperparâmetros. Vamos definir uma função chamada K-Means Objective, que receberá uma tentativa de ajuste dos hiperparâmetros. Vamos ajustar dois hiperparâmetros: o número de clusters (K) e a métrica de distância (Euclidiana e Minkowski). Em seguida, vamos instanciar o K-Means com esses hiperparâmetros e treinar o modelo. Calcularemos o Silhouette Score usando a função pairwise distance e os rótulos do K-Means. Por fim, criaremos um estudo do Optuna e rodaremos o experimento para encontrar a melhor configuração de hiperparâmetros.

In [73]:
# optuna para Otimização de Hiperparâmetros
def kmeans_objective(trial):

    # definir hiperparâmetros
    n_clusters = trial.suggest_int('n_clusters', 3, 10)
    distance_metric = trial.suggest_categorical(
        'distance_metric', ['euclidean', 'minkowski'])

    # criar modelo
    kmeans_model = KMeans(n_clusters=n_clusters, random_state=51)

    # treinar modelo
    kmeans_model.fit(X=df_clients_transformed)

    # calcular Silhouette Score
    distances = pairwise_distances(
        X=df_clients_transformed, metric=distance_metric)
    silhouette_avg = silhouette_score(X=distances, labels=kmeans_model.labels_)

    return silhouette_avg

In [74]:
# criar estudo do optuna
search_space = {'n_clusters': range(3, 10), 'distance_metric': [
    'euclidean', 'minkowski']}
sampler = optuna.samplers.GridSampler(search_space=search_space)
kmeans_study = optuna.create_study(
    study_name='kmeans_study', direction='maximize', sampler=sampler)

# executar o estudo
kmeans_study.optimize(func=kmeans_objective, n_trials=100)

[I 2024-05-11 02:30:36,451] A new study created in memory with name: kmeans_study
[I 2024-05-11 02:30:36,483] Trial 0 finished with value: 0.3251346292525582 and parameters: {'n_clusters': 5, 'distance_metric': 'minkowski'}. Best is trial 0 with value: 0.3251346292525582.
[I 2024-05-11 02:30:36,514] Trial 1 finished with value: 0.18452348191367443 and parameters: {'n_clusters': 7, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.3251346292525582.
[I 2024-05-11 02:30:36,537] Trial 2 finished with value: 0.14731572416665717 and parameters: {'n_clusters': 8, 'distance_metric': 'euclidean'}. Best is trial 0 with value: 0.3251346292525582.
[I 2024-05-11 02:30:36,560] Trial 3 finished with value: 0.3847879496502923 and parameters: {'n_clusters': 4, 'distance_metric': 'euclidean'}. Best is trial 3 with value: 0.3847879496502923.
[I 2024-05-11 02:30:36,581] Trial 4 finished with value: 0.22465301772488028 and parameters: {'n_clusters': 6, 'distance_metric': 'minkowski'}. Best is 

### Bloco 3 - Análise de Resutados

#### Analisando Resultados do KMeans

Neste vídeo, vamos analisar os resultados do Optuna e treinar um modelo com os melhores parâmetros encontrados. Primeiro, vamos guardar a melhor configuração em uma variável chamada "best_params". Em seguida, vamos criar o modelo usando esses melhores parâmetros e ajustá-lo aos dados. Calcularemos o score de silhueta para avaliar o desempenho do modelo. O Optuna determinou que o número de clusters ideal é 3, a métrica de distância escolhida foi a euclidiana e o score de silhueta foi de 0.44. Agora, vamos atribuir os clusters aos registros do nosso dataset e visualizar os primeiros registros com a coluna de cluster atualizada. No próximo vídeo, iremos explorar esses clusters em uma visualização de dados.

In [75]:
best_params = kmeans_study.best_params

best_kmeans = KMeans(n_clusters=best_params['n_clusters'], random_state=51)
best_kmeans.fit(X=df_clients_transformed)

distances = pairwise_distances(X=df_clients_transformed, metric=best_params['distance_metric'])
best_silhouette = silhouette_score(X=distances, labels=best_kmeans.labels_)

print(f'k (número de clusters): {best_params["n_clusters"]}')
print(f'Métrica de Distância Selecionada: {best_params['distance_metric']}')
print(f'Silhouette Score: {best_silhouette}')

k (número de clusters): 3
Métrica de Distância Selecionada: euclidean
Silhouette Score: 0.4445458290999088


In [76]:
# criar coluna com clusters escolhidos
df_clients['cluster'] = best_kmeans.labels_
df_clients.head()

,atividade_economica,faturamento_mensal,numero_de_funcionarios,localizacao,idade,inovacao,cluster
0,Comércio,713109.95,12,Rio de Janeiro,6,1,0
1,Comércio,790714.38,9,São Paulo,15,0,0
2,Comércio,1197239.33,17,São Paulo,4,9,1
3,Indústria,449185.78,15,São Paulo,6,0,0
4,Agronegócio,1006373.16,15,São Paulo,15,8,1


#### Visualização Decisões do KMeans

Neste vídeo, vamos analisar os clusters gerados pelo algoritmo e tentar entender os critérios utilizados para organizá-los. Faremos cruzamentos de dados, como idade e faturamento mensal, e criaremos visualizações para identificar padrões. Observamos que não há um padrão claro baseado no faturamento, mas encontramos um padrão interessante relacionado à inovação. Empresas com níveis de inovação baixos foram agrupadas em um cluster, independentemente do faturamento, enquanto empresas com níveis de inovação moderados e avançados foram agrupadas em outros clusters. Isso nos revela a possibilidade de personalizar o atendimento aos clientes com base em seu nível de inovação. Salvamos o modelo e o pipeline para uso futuro na aplicação batch.

In [77]:
# Cruzar idade e faturamento em um grafico de dispersão 
px.scatter(data_frame=df_clients, x='idade', y='faturamento_mensal', color='cluster')

In [78]:
# Cruzar inovação e faturamento em um grafico de dispersão
px.scatter(df_clients, x='inovacao', y='faturamento_mensal', color='cluster')

In [79]:
# cruzar numero de funcionários e faturamento em um grafico de dispersão
px.scatter(df_clients, x='numero_de_funcionarios', y='faturamento_mensal', color='cluster')

### Bloco 4 - Entrega do Modelo com App Batch

#### Aplicação Batch com Gradio

Nesta aula, desenvolvemos uma aplicação simples no Gradle. Utilizamos um algoritmo de clusterização não supervisionado para transformar os dados em um pipeline separado. Não fizemos predições, apenas trouxemos uma estrutura com o algoritmo. Carregamos o modelo, definimos uma função chamada "clustering" que recebe um arquivo CSV, lê o arquivo, transforma os dados para o formato necessário, treina o modelo e adiciona a coluna de cluster no DataFrame. Salvamos o DataFrame em um arquivo CSV e retornamos o nome desse arquivo. Criamos uma interface para chamar a função e rodamos a aplicação. Agora podemos fazer upload de um arquivo, que será processado e gerará um arquivo de saída com os clusters.

In [80]:
# salvar modelo
joblib.dump(value=best_kmeans, filename='../models/model_clustering_clients.pkl')

# salvar pipeline
joblib.dump(value=preprocessor, filename='../pipelines/pipeline_clustering_clients.pkl')

FileNotFoundError: [Errno 2] No such file or directory: '../pipelines/pipeline_clustering_clients.pkl'

In [ ]:
model = joblib.load(filename='../models/model_clustering_clients.pkl')
preprocessor = joblib.load(filename='../pipelines/pipeline_clustering_clients.pkl')

def clustering(file) -> str:
    
    # carregar CSV para Dataframe
    df_companies = pd.read_csv(filepath_or_buffer=file.name)
    
    # transformar os dados do df para o formato de entrada do modelo
    df_clients_transformed = preprocessor.fit_transform(X=df_companies)
    
    # treinar o modelo
    model.fit(X=df_clients_transformed)
    
    # adicionar coluna com cluster
    df_companies['cluster'] = model.labels_
    
    # exportar para csv
    df_companies.to_csv(path_or_buf='../datasets/processed/companies_clusters.csv', index=False)
    
    return '../datasets/processed/companies_clusters.csv'

In [ ]:
# criar interface
app = gr.Interface(fn=clustering, inputs=gr.File(file_types=[".csv"]), outputs="file")

# executar
app.launch()

Running on local URL:  http://127.0.0.1:7862

To create a public link, set `share=True` in `launch()`.
